In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기

In [ ]:
DATA_PATH = "/content/drive/MyDrive/멋쟁이사자처럼/Final_Project/dataset/"
SEED = 42

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [ ]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_1111.csv")

train_ft.shape, test_ft.shape

((14940, 1470), (12225, 1470))

# 로그 변환 (Log Transformation):

- 주로 값의 범위가 넓을 때 사용하는 방법입니다. 값이 큰 쪽에 치우쳐 있거나 비대칭적인 분포를 가진 데이터를 변환해 분포를 균등하게 만듭니다.

- 예를 들어, 수입이나 소득처럼 큰 값들이 분포에 영향을 미치는 경우, 로그 변환을 하면 값의 스케일을 줄이고 분포를 더 대칭적으로 만들 수 있습니다.

- 사용법:
𝑥
x의 값이 0보다 커야 로그를 사용할 수 있습니다. 예를 들어, np.log(x) 또는 10을 밑으로 사용하고 싶다면 np.log10(x)를 사용합니다.

- 문제 정의: 데이터가 단순하고 예측력이 낮을 때.

- 비선형 변환:
    - log, sqrt, exp 등의 비선형 변환으로 데이터 스케일링.

In [ ]:
# 수치형 피처만 필터링
numeric_df = train_ft.select_dtypes(include=['number'])

# 이상치 개수와 왜도를 저장할 딕셔너리
outlier_stats = {}

# 각 수치형 피처에서 이상치 개수와 왜도를 계산하여 딕셔너리에 저장
for column in numeric_df.columns:
    # IQR (Interquartile Range) 계산
    Q1 = numeric_df[column].quantile(0.25)
    Q3 = numeric_df[column].quantile(0.75)
    IQR = Q3 - Q1

    # 이상치의 기준점 설정 (IQR의 1.5배 초과/미만으로 간주)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 이상치 개수 계산
    outliers = numeric_df[(numeric_df[column] < lower_bound) | (numeric_df[column] > upper_bound)][column]
    outlier_count = len(outliers)

    # 왜도 계산
    skewness = numeric_df[column].skew()

    # 딕셔너리에 이상치 개수와 왜도 저장
    outlier_stats[column] = {'outlier_count': outlier_count, 'skewness': skewness}

# 이상치 개수가 747개 이상인 피처들만 필터링 --> 전체 데이터의 5% 이상
filtered_outliers = {k: v for k, v in outlier_stats.items() if v['outlier_count'] >= 747}

# 이상치 개수가 많은 피처를 내림차순으로 정렬
sorted_outliers = sorted(filtered_outliers.items(), key=lambda x: x[1]['outlier_count'], reverse=True)

# 결과 출력
for feature, stats in sorted_outliers:
    print(f"Feature: {feature}, Outlier Count: {stats['outlier_count']}, Skewness: {stats['skewness']:.2f}")

Feature: A202000지점_구매금액, Outlier Count: 3738, Skewness: 11.05
Feature: pv_대분류_여성정장_구매횟수, Outlier Count: 3704, Skewness: 9.54
Feature: pv_대분류_아동_스포츠_구입금액비중, Outlier Count: 3667, Skewness: 4.91
Feature: 대분류_여성캐쥬얼_구매주기, Outlier Count: 3655, Skewness: 2.74
Feature: pv_대분류_골프_유니캐쥬얼_구매횟수, Outlier Count: 3642, Skewness: 5.33
Feature: pv_대분류_골프_유니캐쥬얼_총구매금액, Outlier Count: 3586, Skewness: 11.53
Feature: pv_대분류_골프_유니캐쥬얼_평균구매금액, Outlier Count: 3586, Skewness: 5.70
Feature: pv_대분류_골프_유니캐쥬얼_구입금액비중, Outlier Count: 3586, Skewness: 4.30
Feature: pv_대분류_아동_스포츠_평균구매금액, Outlier Count: 3548, Skewness: 27.76
Feature: pv_대분류_여성정장_총구매금액, Outlier Count: 3542, Skewness: 10.41
Feature: pv_대분류_여성정장_평균구매금액, Outlier Count: 3542, Skewness: 9.11
Feature: pv_대분류_여성정장_구입금액비중, Outlier Count: 3542, Skewness: 3.83
Feature: pv_대분류_아동_스포츠_총구매금액, Outlier Count: 3532, Skewness: 14.10
Feature: 대분류_케주얼_구두_아동_구매주기, Outlier Count: 3451, Skewness: 3.15
Feature: pv_대분류_영플라자_구매횟수, Outlier Count: 3426, Skewness: 8.88
Feature: pv_대분류

In [ ]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 로그 변환
for feature in filtered_outliers:
    # 훈련 데이터에서 0 또는 음수 값 처리 후 로그 변환
    train_ft[feature] = train_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

# 테스트 데이터에 대해서도 훈련 데이터에서 동일한 방식으로 로그 변환
for feature in filtered_outliers:
    # 테스트 데이터에 대해 훈련 데이터에서 계산된 방식대로 처리
    test_ft[feature] = test_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

# 제곱근 변환 (Square Root Transformation):

- 값이 큰 범위에 몰려 있을 때 이를 줄여주기 위해 사용됩니다. 로그보다는 덜 극단적인 변환으로, 데이터의 분포를 완만하게 만듭니다.

- 예를 들어, 인구 수 같은 양수 데이터에서 제곱근을 적용하면 값의 분포를 균형 잡힌 형태로 만들 수 있습니다.

- 사용법: 양수 데이터를 대상으로 하며, np.sqrt(x)를 통해 제곱근을 적용할 수 있습니다.

## 제곱근 변환만 적용

In [ ]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 제곱근 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (train_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        train_ft[feature] = train_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에도 제곱근 변환
        train_ft[feature] = train_ft[feature].apply(lambda x: np.sqrt(np.abs(x)) if x >= 0 else 0)

# 테스트 데이터에 대해서도 동일한 방식으로 제곱근 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (test_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        test_ft[feature] = test_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에도 제곱근 변환
        test_ft[feature] = test_ft[feature].apply(lambda x: np.sqrt(np.abs(x)) if x >= 0 else 0)

 ## 양수로만 이루어진 피처는 제곱근 변환 이외는 로그 변환

In [ ]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (train_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        train_ft[feature] = train_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에는 로그 변환
        train_ft[feature] = train_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

# 테스트 데이터에 대해서도 동일한 방식으로 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (test_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        test_ft[feature] = test_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에는 로그 변환
        test_ft[feature] = test_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

## 피처 값 중 양수는 제곱근 변환 이외에는 로그 변환

In [ ]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 변환 적용
for feature in filtered_outliers:
    # 양수인 값은 제곱근 변환, 0 이하의 값은 로그 변환
    train_ft[feature] = train_ft[feature].apply(
        lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환
    )
    # 변환 후 NaN 값이 있을 경우 0으로 대체
    train_ft[feature] = train_ft[feature].fillna(0)

# 테스트 데이터에 대해서도 동일한 방식으로 변환 적용
for feature in filtered_outliers:
    # 양수인 값은 제곱근 변환, 0 이하의 값은 로그 변환
    test_ft[feature] = test_ft[feature].apply(
        lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환
    )
    # 변환 후 NaN 값이 있을 경우 0으로 대체
    test_ft[feature] = test_ft[feature].fillna(0)

<ipython-input-11-93a17ec4e807>:8: RuntimeWarning: invalid value encountered in log
  lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환
<ipython-input-11-93a17ec4e807>:17: RuntimeWarning: invalid value encountered in log
  lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환


# 지수 변환 (Exponential Transformation):

- 주로 값을 확장해야 하는 경우 사용합니다. 예를 들어, 분포가 너무 작게 몰려 있을 때 지수 변환을 통해 데이터의 차이를 크게 만들어 모델이 차이를 더 잘 학습하게 할 수 있습니다.

- 사용법: np.exp(x)로 변환합니다. 다만, 값이 너무 커질 수 있으니 주의가 필요합니다.

# Box-Cox 변환 (Box-Cox Transformation)

- 데이터 분포가 치우쳐져 있는 경우, 특히 정규 분포에 가깝게 만들기 위해 사용하는 변환 방법입니다.

- Box-Cox는 **람다(λ)**라는 파라미터를 사용해 최적의 변환을 찾습니다. 이 변환을 통해 데이터 분포를 정규에 가깝게 조정할 수 있습니다.

- 사용법: SciPy의 boxcox 함수를 사용하며, 데이터는 양수여야 합니다.

# 역변환

- 데이터의 값이 클수록 작은 값으로 줄어들게 만드는 방식으로, 비대칭 분포를 조정할 때 유용합니다.

- 사용법: 역수를 취해 변환하며, 1/x 형태로 사용할 수 있습니다.

# 피처 공학 및 스케일링

In [ ]:
# ID 제거
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [ ]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,238


In [ ]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.4 MB/s eta 0:00:00


In [ ]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft["주구매지점"])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft["주구매지점"])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [ ]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [ ]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

In [ ]:
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.255374,0.002987,0.257728,-0.257728,0.428635,-0.128919,0.346813,0.001191,0.838272,0.338186,...,-0.162079,-0.352780,-0.276145,-0.259675,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.402970,-0.356452,-1.008554,1.008554,0.294145,0.554897,0.398608,-0.390607,0.620171,-0.552996,...,2.389509,-0.352780,-0.276145,3.890999,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.911468,-0.869935,0.036742,-0.036742,0.718373,1.388325,0.410273,-0.514333,-0.304527,-0.059266,...,-0.162079,-0.352780,-0.276145,-0.259675,-0.604571,-0.623175,1.632592,-0.468181,-0.869072,1.064726
3,3.007893,-1.023980,-0.080558,0.080558,0.873482,2.358551,0.401120,-0.327474,0.008592,-0.135636,...,10.044273,-0.352780,3.562547,-0.259675,1.654066,-0.623175,-0.612523,-0.468181,1.515861,-0.248364
4,0.722054,-0.613193,0.302875,-0.302875,0.620560,0.657384,0.362380,1.707410,-0.130285,-0.821561,...,-0.162079,2.905522,3.691907,-0.259675,1.654066,-0.623175,-0.612523,-0.468181,-0.847712,1.064726


In [ ]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [ ]:
target = train_target["target"]

# 점수 확인

In [ ]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

# 이상치 기준 상위 500개 피처 로그변환 -> 0.7227222269087612
# 전체 데이터 중 이상치가 5% 이상인 피처에 대한 로그변환 -> 0.7227222269087612
# 제곱근 변환 -> 0.7209092345457006
# 양수로만 이뤄진 수치형 데이터는 제곱근 변환 + 나머지는 로그 변환 -> 0.7227222269087612
# 각 피처의 값 중 양수는 제곱근 변환, 나머지 0과 음수일 경우 로그 변환 -> 0.7234808173962948 O

Mean F1 Macro Score: 0.7234808173962948


----

In [ ]:
train_ft.to_csv("train_common_v3.3_피처삭제X_log_sqrt.csv",index=False)
test_ft.to_csv("test_common_v3.3_피처삭제X_log_sqrt.csv",index=False)

# Optuna

In [ ]:
%pip install optuna

In [ ]:
import optuna
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## XGBOOST

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier

class Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = StratifiedKFold(5, shuffle= True, random_state= self.seed)

    def __call__(self, trial):
        hp = {  "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "gamma": trial.suggest_float("gamma", 0, 5)  }
        model = XGBClassifier(**hp, random_state= self.seed)
        score = cross_val_score(model, self.x, self.y, cv= self.cv, scoring= "f1_macro", n_jobs= -1).mean()
        return score

sampler = optuna.samplers.TPESampler(seed= SEED)

study = optuna.create_study(direction= "maximize", sampler= sampler)

objective_func = Objective(train_ft, target, SEED)
study.optimize(objective_func, n_trials= 100)

[I 2024-11-12 08:24:56,307] A new study created in memory with name: no-name-b533bcd4-7568-49f3-b0f1-564ac63a6a81
[I 2024-11-12 08:33:12,712] Trial 0 finished with value: 0.6886174135705478 and parameters: {'n_estimators': 400, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 0.6886174135705478.
[I 2024-11-12 08:44:47,172] Trial 1 finished with value: 0.7115088283992492 and parameters: {'n_estimators': 900, 'learning_rate': 0.07725378389307355, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 1.0616955533913808}. Best is trial 1 with value: 0.7115088283992492.
[I 2024-11-12 08:48:33,829] Trial 2 finished with value: 0.706431646271415 and parameters: {'n_estimators': 200, 'learning_rate': 0.018659959624904916, 'max_depth': 5, 'min_child_weight': 6, 'subsample':

In [ ]:
xgb_best_params = {'n_estimators': 900,
                   'learning_rate': 0.021734683976721573,
                   'max_depth': 6,
                   'min_child_weight': 5,
                   'subsample': 0.7039147285447092,
                   'colsample_bytree': 0.5823954097077503,
                   'gamma': 2.3107011128410493}
# 0.7231554675553863

## LGBM

In [ ]:
from lightgbm import LGBMClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
def objective(trial):
    hp = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),               # 리프 노드의 수
        "max_depth": trial.suggest_int("max_depth", 3, 15),                   # 최대 트리 깊이
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),  # 학습률
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),      # 부스팅 단계 수
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),  # 최소 데이터 수
        "subsample": trial.suggest_float("subsample", 0.6, 1.0, step=0.1),    # 샘플링 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1),  # 피처 샘플링 비율
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),  # L1 규제
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True) # L2 규제
    }

    model = LGBMClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=50)

[I 2024-11-12 15:50:00,454] A new study created in memory with name: no-name-ad2e0a12-ea88-411c-8784-d3a8d3e953fb
[I 2024-11-12 15:54:48,972] Trial 0 finished with value: 0.7108780019097494 and parameters: {'num_leaves': 69, 'max_depth': 15, 'learning_rate': 0.1205712628744377, 'n_estimators': 600, 'min_child_samples': 19, 'subsample': 0.6, 'colsample_bytree': 0.6, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598}. Best is trial 0 with value: 0.7108780019097494.
[I 2024-11-12 15:56:52,862] Trial 1 finished with value: 0.7039690030821333 and parameters: {'num_leaves': 112, 'max_depth': 3, 'learning_rate': 0.2708160864249968, 'n_estimators': 900, 'min_child_samples': 25, 'subsample': 0.6, 'colsample_bytree': 0.6, 'reg_alpha': 5.472429642032198e-06, 'reg_lambda': 0.00052821153945323}. Best is trial 0 with value: 0.7108780019097494.
[I 2024-11-12 15:57:57,053] Trial 2 finished with value: 0.7142086215730521 and parameters: {'num_leaves': 76, 'max_depth': 6, 'learning_rat

In [ ]:
lgbm_best_params = {'num_leaves': 33,
                    'max_depth': 11,
                    'learning_rate': 0.01944354088862471,
                    'n_estimators': 800,
                    'min_child_samples': 65,
                    'subsample': 0.8,
                    'colsample_bytree': 0.8,
                    'reg_alpha': 3.6529269182968447e-07,
                    'reg_lambda': 3.3190277841665685e-07}

# cv = 0.7252597292985927

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(**lgbm_best_params, random_state=SEED)
model.fit(train_ft, target)

# pred = model.predict(test_ft)
# submit["target"] = pred
# submit.to_csv("log_sqrt_LGBM.csv", index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.284892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 135303
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 1185
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
proba = model.predict_proba(test_ft)[:,1]
submit["target"] = proba
submit.to_csv("log_sqrt_LGBM.csv", index=False)